In [4]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import display
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error,mean_squared_log_error

In [5]:
df = pd.read_csv("TRAIN.csv",parse_dates=['Date']) # Reading File
print(df.shape)
df.head()

(188340, 10)


,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [6]:
df['Weekday'] = df.Date.apply(lambda x:x.weekday())  # Extracting weekday from date
df['Weekday']

0         0
1         0
2         0
3         0
4         0
         ..
188335    4
188336    4
188337    4
188338    4
188339    4
Name: Weekday, Length: 188340, dtype: int64

In [7]:
df["Store_id"] = df.Store_id/(365/7) # Scaling Store_id column

In [8]:
y = df[['Sales']]  # Target
used_cols = ['Store_id','Store_Type','Location_Type','Region_Code', 'Holiday','Discount','Weekday']  # Predictor
X = df[used_cols]
X = pd.get_dummies(X,drop_first=True)  # Dummification of categorical variables
X

,Store_id,Holiday,Weekday,Store_Type_S2,Store_Type_S3,Store_Type_S4,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R2,Region_Code_R3,Region_Code_R4,Discount_Yes
0,0.019178,1,0,0,0,0,0,1,0,0,0,0,0,1
1,4.852055,1,0,0,0,1,1,0,0,0,0,0,0,1
2,4.832877,1,0,0,1,0,1,0,0,0,0,0,0,1
3,4.813699,1,0,1,0,0,0,1,0,0,0,0,0,1
4,4.794521,1,0,1,0,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,2.857534,1,4,1,0,0,0,1,0,0,1,0,0,1
188336,2.934247,1,4,0,0,1,1,0,0,0,0,0,0,0
188337,2.953425,1,4,0,0,0,0,1,0,0,1,0,0,0
188338,2.972603,1,4,0,1,0,0,0,0,0,1,0,0,1


In [9]:
X_train, X_cv, y_train, y_cv = tts(X,y,test_size=0.3,random_state=1245)  # Train and cross validation split
X_train.shape,y_train.shape,X_cv.shape,y_cv.shape

((131838, 14), (131838, 1), (56502, 14), (56502, 1))

In [10]:
from sklearn.ensemble import GradientBoostingRegressor


# making instance of gbr
gbr = GradientBoostingRegressor(n_estimators=183,criterion='mse',
                                max_depth=7,init=None,random_state=12345,max_features=9
                                )

# fitting data in grb
gbr.fit(X_train,y_train.Sales.values)

# Prediction of cross validation data
yhat_cv = gbr.predict(X_cv)

# mean square log error
msle = mean_squared_log_error(y_cv,yhat_cv)
msle

0.08890447465592752

In [11]:
# Training Gradient Boosting Regressor on all data
gbr = GradientBoostingRegressor(n_estimators=183,criterion='mse',
                                max_depth=7,init=None,random_state=12345,max_features=9
                                )
gbr.fit(X=X,y=y.Sales.values)

yhat = gbr.predict(X)
msle = mean_squared_log_error(y_true=y,y_pred=yhat)
msle

0.08786736935282981

In [12]:

# Loading test data
test_df= pd.read_csv('TEST_FINAL.csv',parse_dates=['Date'])
test_df['Store_id'] = test_df.Store_id/(365/7)  # Scaling Store_id column

test_id = test_df.ID # Storing ID column for later use

test_df['Weekday'] = test_df.Date.apply(lambda x:x.weekday()) # Weekday calculation for test data

test_df = test_df[['Store_id','Store_Type','Location_Type','Region_Code',
                   'Holiday','Discount','Weekday']]  # Predictors

test_df = pd.get_dummies(test_df,drop_first=True) # Dummification of categorical variables

In [13]:
prediction_on_test = gbr.predict(test_df)
prediction_on_test.shape, test_id.shape

((22265,), (22265,))

# Saving Result in Result.csv
#Prediction = np.squeeze(Prediction)
submission = pd.DataFrame({
                           "ID": test_id,
                           "Sales": prediction_on_test
                           })
submission.to_csv('Resultcopied.csv', index=True)